<h1>Estudo da Covid-19 no Brasil</h1>
<b>Éverton Bin</b><br>
<b>Junho, 2020</b>

<h2>Índice</h2>
<ol>
    <a href = '#intro'><li><b>Introdução</b></li></a>
    <a href = '#carrega'><li><b>Carregando os Dados</b></li></a>
    <a href = '#analise'><li><b>Análise Exploratória</b></li></a>
    <a href = '#modelo'><li><b>Criação dos Modelos Preditivos</b></a>
        <ol>
            <a href = '#modelosp'><li><b>Modelos de Regressão: São Paulo - SP</b></li></a>
            <a href = '#modelorj'><li><b>Modelos de Regressão: Rio de Janeiro - RJ</b></li></a>
            <a href = '#modelobl'><li><b>Modelos de Regressão: Belém - PA</b></li></a>
            <a href = '#modelopa'><li><b>Modelos de Regressão: Porto Alegre - RS</b></li></a>
        </ol>
    <a href = '#prev'><li><b>Previsões</b></li></a>
        <ol>
            <a href = '#prevsp'><li><b>Prevendo as Mortes em São Paulo - SP</b></li></a>
            <a href = '#prevrj'><li><b>Prevendo as Mortes no Rio de Janeiro - RJ</b></li></a>
            <a href = '#prevbl'><li><b>Prevendo as Mortes em Belém - PA</b></li></a>
            <a href = '#prevpa'><li><b>Prevendo as Mortes em Porto Alegre - RS</b></li></a>
        </ol>
    <a href = '#conclusao'><li><b>Conclusão</b></li></a>
</ol>

<h2 id = intro>1- Introdução</h2>

<p>
    Este projeto tem como objetivo fazer uma análise dos dados sobre a pandemia da Covid-19 no Brasil, sobretudo em algumas cidades brasileiras de interesse. Os dados utilizados neste notebook são disponibilizados no Kaggle e podem ser acessados no seguinte endereço: <a href = "https://www.kaggle.com/unanimad/corona-virus-brazil/data?select=brazil_population_2019.csv"> Coronavirus - Brazil </a>. Após esta breve análise, o objetivo é construir um modelo de regressão capaz de prever o número de mortes causadas pelo vírus em datas futuras.
</p>
<p>
    O arquivo de referência para este estudo, "brazil_covid19_cities.csv", apresenta os seguintes dados sobre a pandemia no país:
    <li><i>date</i>: data de confirmação dos casos;</li>
    <li><i>state</i>: estado brasileiro de referência dos casos;</li>
    <li><i>name</i>: nome da cidade de referência dos casos;</li>
    <li><i>code</i>: código da cidade;</li>
    <li><i>cases</i>: número de casos confirmados;</li>
    <li><i>deaths</i>: número de mortes confirmadas pelo Covid19.</li>
</p>
<p>
    Este dataset será analisado juntamente com o dataset "brazil_population_2019.csv", também disponibilizado na página indicada anteriormente, para que se possa avaliar a situação dos casos proporcionalmente à população da cidade de referência.</p>

<h2 id = carrega>2- Carregando os Dados</h2>

In [ ]:
# Pacotes utilizados:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import datetime
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [ ]:
# Leitura dos arquivos:
covid_br = pd.read_csv('../input/covid19-br/brazil_covid19_cities.csv', parse_dates = ['date'])
cities_pop = pd.read_csv('../input/covid19-br/brazil_population_2019.csv')

In [ ]:
# Verificando o formato das variáveis:
covid_br.dtypes

In [ ]:
# Observando as primeiras linhas dos datasets:
covid_br.head()

In [ ]:
# Alterando a nomeclatura de cada estado para o seu respectivo 'UF':
estado_uf = {'Acre': 'AC', 'Alagoas': 'AL', 'Amapá': 'AP', 'Amazonas': 'AM', 'Bahia': 'BA', 'Ceará': 'CE',
              'Distrito Federal': 'DF', 'Espírito Santo': 'ES', 'Goiás': 'GO', 'Maranhão': 'MA', 'Minas Gerais': 'MG',
              'Mato Grosso do Sul': 'MS', 'Mato Grosso': 'MT', 'Pará': 'PA', 'Paraíba': 'PB', 'Pernambuco': 'PE',
              'Piauí': 'PI', 'Paraná': 'PR', 'Rio de Janeiro': 'RJ', 'Rio Grande do Norte': 'RN', 'Rondônia': 'RO',
              'Roraima': 'RR', 'Rio Grande do Sul': 'RS', 'Santa Catarina': 'SC', 'Sergipe': 'SE', 'São Paulo': 'SP',
              'Tocantins': 'TO'}

covid_br = covid_br.replace({'state': estado_uf})

# Criando a variável 'city', concatenando a cidade e o 'UF' do seu respectivo estado de referência:
covid_br['city'] = covid_br['name'] + ' - ' + covid_br['state']

# Selecionando apenas as capitais de cada estado:
capitais = ['Aracaju - SE', 'Belém - PA', 'Belo Horizonte - MG', 'Boa Vista - RR', 'Brasília - DF', 'Campo Grande - MS',
            'Cuiabá - MT', 'Curitiba - PR', 'Florianópolis - SC', 'Fortaleza - CE', 'Goiânia - GO', 'João Pessoa - PB',
            'Macapá - AP', 'Maceió - AL', 'Manaus - AM', 'Natal - RN', 'Palmas - TO', 'Porto Alegre - RS',
            'Porto Velho - RO', 'Recife - PE', 'Rio Branco - AC', 'Rio de Janeiro - RJ', 'Salvador - BA', 'São Luís - MA',
            'São Paulo - SP', 'Teresina - PI', 'Vitória - ES']

covid_cap_br = covid_br[covid_br.city.isin(capitais)]

# Deletando as colunas 'state' e 'name' do dataset:
covid_cap_br = covid_cap_br.drop(columns = ['state', 'name'])

covid_cap_br.head()

In [ ]:
# Incorporando o número populacional ao dataset com informações sobre a Covid-19:
covid_cap_br = covid_cap_br.rename(columns = {'code' : 'city_code'})
cities_pop = cities_pop[['city_code', 'population']]
covid_cap_br = pd.merge(covid_cap_br, cities_pop, on = 'city_code')
covid_cap_br.head()

<h2 id = analise>3- Análise Exploratória</h2>

<p>
    Serão criadas novas colunas para representar o número de casos e mortes por 100 mil habitantes para que, posteriormente, as comparações possam ser feitas de maneira proporcional.
</p>

In [ ]:
covid_atual = covid_cap_br[covid_cap_br['date'] == max(covid_cap_br['date'])]

covid_atual['cases_rate'] = round(((covid_atual['cases'] / covid_atual['population'])*100000), 1)

covid_atual['deaths_rate'] = round(((covid_atual['deaths'] / covid_atual['population'])*100000), 1)

covid_atual['deaths_prop'] = round(((covid_atual['deaths'] / covid_atual['cases'])*100), 1)

covid_atual.head()

In [ ]:
# Ordenando o dataset por número de mortes:
covid_deaths = covid_atual.sort_values(by = ['deaths'], ascending = False)

# Ordenando o dataset por número de mortes por 100 mil habitantes:
covid_deaths_rate = covid_atual.sort_values(by = ['deaths_rate'], ascending = False)

# Ordenando o dataset por número de casos confirmados por 100 mil habitantes:
covid_cases_rate = covid_atual.sort_values(by = ['cases_rate'], ascending = False)

# Criando variáveis que indicam a primeira e última datas apresentadas no dataset:
ult_date = max(covid_cap_br['date']).strftime('%d/%m/%Y')
min_date = min(covid_cap_br['date']).strftime('%d/%m/%Y')

# Criando o plot:
fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (16, 5))

sns.barplot(x = 'city', y = 'deaths', data = covid_deaths[0:5], ci = None,
            palette = 'Blues', ax = ax[0]).set_title('MORTES TOTAIS ATÉ '+ ult_date)
ax[0].set_xlabel('Cidade')
ax[0].set_ylabel('Número de mortos')

sns.barplot(x = 'city', y = 'deaths_rate', data = covid_deaths_rate[0:5], ci = None,
            palette = 'Blues', ax = ax[1]).set_title('MORTES POR 100 MIL HABITANTES ATÉ '+ ult_date)
ax[1].set_xlabel('Cidade')
ax[1].set_ylabel('Número de mortos por 100 mil hab.')

fig.show()

<p>
    É possível observar que, se for considerado unicamente o número de mortos, as cidades de <b>São Paulo</b> e <b>Rio de Janeiro</b> são os destaques negativos no país, sendo responsáveis pelo maior número de mortes entre as capitais brasileiras.
</p>
<p>
    No entanto, se considerarmos o número de mortes pelo Covid-19 relativamente à população total de cada uma das cidades, São Paulo e Rio de Janeiro dão lugar às cidades de <b>Belém</b> e <b>Fortaleza</b> como sendo as capitais com mais mortes por 100 mil habitantes no Brasil, sendo que <b>Belém</b> está na casa de 90 mortes por 100 mil habitantes, enquanto <b>Fortaleza</b> apresenta número próximo a 80 mortes por 100 mil habitantes.
</p>

In [ ]:
# Ordenando o dataset por percentagem de mortos entre os casos confirmados:
covid_deaths_prop = covid_atual.sort_values(by = ['deaths_prop'], ascending = False)

# Criando o plot:
fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (16, 5))

sns.barplot(x = 'city', y = 'deaths_rate', data = covid_deaths_rate[0:5], ci = None,
            palette = 'Blues', ax = ax[0]).set_title('MORTES POR 100 MIL HABITANTES ATÉ '+ ult_date)
ax[0].set_xlabel('Cidade')
ax[0].set_ylabel('Número de mortos por 100 mil hab.')

sns.barplot(x = 'city', y = 'deaths_prop', data = covid_deaths_prop[0:5], ci = None,
            palette = 'Blues', ax = ax[1]).set_title('PERCENTAGEM DE MORTOS ENTRE CASOS CONFIRMADOS ATÉ '+ ult_date)
ax[1].set_xlabel('Cidade')
ax[1].set_ylabel('Percentagem de mortos entre confirmados')

fig.show()

<p>
    Observando os gráficos comparativos acima, pode-se perceber que outros fatores, além da proporção de mortes relativa ao tamanho da população, parecem afetar a percentagem de mortos entre os casos confirmados. Embora <b>Belém</b> seja a capital com a maior proporção de mortos para 100 mil habitantes, é o <b>Rio de Janeiro</b> que apresenta a maior proporção de mortes entre os casos confirmados, número este que supera os 12%. <b>Belém</b> apresenta número próximo ao número apresentado pelo Rio de Janeiro, porém levemente menor.
</p>
<p>
    Vamos avaliar também quais são as capitais que se destacam positivamente no país, utilizando como ponto de vista o número total de mortes, o número de mortes por 100 mil habitantes e também a percentagem de mortos entre os casos confirmados:
</p>

In [ ]:
# Criando o plot:
fig, ax = plt.subplots(nrows = 2, ncols = 2, figsize = (17, 10))

sns.barplot(x = 'city', y = 'deaths', data = covid_deaths[22:], ci = None,
            palette = 'Blues', ax = ax[0,0]).set_title('MORTES TOTAIS ATÉ '+ ult_date)
ax[0,0].set_xlabel('Cidade')
ax[0,0].set_ylabel('Número de mortos')

sns.barplot(x = 'city', y = 'deaths_rate', data = covid_deaths_rate[22:], ci = None,
            palette = 'Blues', ax = ax[0,1]).set_title('MORTES POR 100 MIL HABITANTES ATÉ '+ ult_date)
ax[0,1].set_xlabel('Cidade')
ax[0,1].set_ylabel('Número de mortos por 100 mil hab.')

sns.barplot(x = 'city', y = 'deaths_prop', data = covid_deaths_prop[22:], ci = None,
            palette = 'Blues', ax = ax[1,0]).set_title('% DE MORTOS ATÉ '+ ult_date)
ax[1,0].set_xlabel('Cidade')
ax[1,0].set_ylabel('Percentagem de mortos entre confirmados')

sns.barplot(x = 'city', y = 'cases_rate', data = covid_cases_rate[22:], ci = None,
            palette = 'Blues', ax = ax[1,1]).set_title('CASOS CONFIRMADOS POR 100 MIL HABITANTES ATÉ '+ ult_date)
ax[1,1].set_xlabel('Cidade')
ax[1,1].set_ylabel('Número de casos confirmados por 100 mil hab.')

fig.show()

<p>
    <b>Porto Alegre</b> está entre as 5 capitais com o menor número absoluto de mortes e também entre as 5 capitais com o menor número de casos confirmados por 100 mil habitantes. Ainda assim, não está entre as 5 capitais com menor número de mortes por 100 mil habitantes ou com a menor percentagem de mortos entre os casos confirmados. Os fatos que justificam esta observação devem ser estudados, no entanto, umas das possíveis razões pode ser o fato de que o Rio Grande do Sul possui uma proporção maior de idosos se comparado aos demais estados do país, sendo a população idosa considerada como um dos grupos com maior risco de desenvolver complicações relacionadas ao vírus.
</p>
<p>
    Já as capitais <b>Cuiabá</b>, <b>Campo Grande</b> e <b>Palmas</b>, do ponto de vista analisado, parecem estar em uma situação mais confortável em relação às demais capitais do país.
</p>

<p>
    Vamos avaliar também a evolução diária de mortes, principalmente nas capitais que se destacaram até o momento nas análises desenvolvidas. Para tal, será criada uma variável que representa a contagem de dias a partir do primeiro registro apresentado no conjunto de dados. Além disso, será criada uma função para facilitar a criação de um gráfico que represente esta evolução diária de mortes, permitindo uma análise visual da tendência apresentada em cada cidade.
</p>

In [ ]:
# Criando uma variável numérica que represente a contagem dos dias a partir da primeira data de referência do dataset:
covid_cap_br['day_count'] = (covid_cap_br['date'] - min(covid_cap_br['date']))
covid_cap_br['day_count'] = pd.to_numeric(covid_cap_br['day_count'].dt.days, downcast='integer')

In [ ]:
# Criando uma função que gera um plot que apresenta a evolução do número de mortes dia a dia:
def deaths_per_day(city_name):
    """Cria um gráfico apresentando as mortes dia a dia.
    
    Args:
    city_name: string indicando a cidade a ser filtrada. A convenção no dataset é 'Cidade - UF'.
    """
    filt_city = covid_cap_br[covid_cap_br['city'] == city_name]
    
    filt_city = filt_city.reset_index(drop = True)
    
    filt_city['deaths_per_day'] = ''
    for i in range(0, len(filt_city.index)):
        if i == 0:
            filt_city['deaths_per_day'][i] = filt_city['deaths'][i]
        else:
            filt_city['deaths_per_day'][i] = filt_city['deaths'][i] - filt_city['deaths'][(i-1)]
    
    fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (16, 5))
    
    sns.barplot(x = 'day_count', y = 'deaths_per_day', data = filt_city, palette = "autumn_r").set(
        xlabel = 'Contagem dos dias desde ' + min_date,
        ylabel = 'Mortes diárias',
        title = 'Evolução diária do número de mortes em ' + city_name)
    fig.show()

In [ ]:
deaths_per_day('São Paulo - SP')

<p>
    Na cidade de <b>São Paulo</b> é possível perceber um padrão no qual os casos de morte reportados diminuem próximo aos finais de semana e aumentam ao longo da semana. É claro também que, após um período de certa estabilidade, os picos de morte apresentam uma tendência de alta.
</p>

In [ ]:
deaths_per_day('Rio de Janeiro - RJ')

<p>
    Parece haver uma tendência de alta no número de mortes na cidade do <b>Rio de Janeiro</b>, tendência esta ainda mais consistente que a apresentada na cidade de São Paulo, onde o aumento de mortes evolui a um passo mais suave. Aparentemente, o segundo mês registrado no dataset representa um aumento sólido no número de mortes no Rio de Janeiro.
</p>

In [ ]:
deaths_per_day('Belém - PA')

<p>
    Em <b>Belém</b>, é possível observar picos maiores ao longo das últimas semanas, indicando que a situação local ainda está piorando.
</p>

In [ ]:
deaths_per_day('Porto Alegre - RS')

<p>
    Como visto anteriormente, <b>Porto Alegre</b> está em uma situação melhor se comparada a outras capitais no país. Ainda assim, o espaçamento de dias sem mortes reportadas diminuiu nos últimos dias, enquanto os picos de mortes apresentam uma tendência de alta. Além disso, a barra 'negativa' para a cidade deve ser referente a algum caso de confirmação excluído posteriormente por duplicidade.
</p>

<h2 id = modelo>4- Criação de modelo de previsão do número de mortes pelo Covid-19</h2>

<p>
    Para a criação deste modelo preditivo, tentaremos prever o número de mortes unicamente pela data, ou seja, esta será a única variável preditora, já que serão criados modelos separados para algumas capitais de interesse. Sendo assim, a variável alvo, cujo valor queremos prever, é o número de mortes total até a data solicitada.
</p>
<p>
    80% do dataset será utilizado para treinar o modelo e os 20% restantes serão utilizados para avaliar a sua performance na previsão do número correto de mortes, dada uma métrica de comparação.
</p>

<p>
    Na seqüência, serão criadas funções para facilitar o desenvolvimento dos modelos para as diferentes cidades de interesse:
</p>

In [ ]:
# Criando função para filtrar cidade e colunas desejadas:
def city_filter(city_name):
    """Filtra o dataset já alterado - neste ponto, covid_cap_br - para a capital de interesse.
    
    Args:
    city_name: string indicando a cidade a ser filtrada. A convenção no dataset é 'Cidade - UF'.
    
    Returns:
    filt_data: dataset contendo os dados referentes à cidade de interesse.
    """
    filt_data = covid_cap_br[covid_cap_br['city'] == city_name]
    filt_data = filt_data[['day_count', 'deaths']]
    return filt_data

# Criando função para plotar gráfico com a evolução do número acumulado de mortes semana a semana:
def deaths_evolution(city_data, city_name):
    """Cria um gráfico de barras com a contagem acumulada das mortes, espaçadas de 7 em 7 dias.
    
    Args:
    city_data: dataset filtrado pela função city_filter();
    city_name: string com o nome da cidade a ser apresentado no gráfico.
    """
    sns.barplot(x = 'day_count', y = 'deaths', data = city_data[city_data['day_count'] % 7 == 0], palette = "Blues").set(
        xlabel = 'Contagem de dias a partir de ' + min_date,
        ylabel = 'Número de mortes',
        title = 'Evolução do número de mortes na cidade de ' + city_name)

In [ ]:
# Criando função para desenvolver, treinar e avaliar o modelo:
def covid_poly_model(city_data, n_degree, seed):
    """Transforma os dados para uma regressão polinomial, cria o modelo de regressão linear e o avalia, juntamente com a
    apresentação de um gráfico que ilustra os dados previstos pelo modelo em relação aos dados originais.
    
    Args:
    city_data: dataset filtrado pela função city_filter();
    n_degree: número que representa o grau da transformação linear a ser aplicada;
    seed: número que permite a reprodução da separação dos dados de treino e de teste para posterior comparação.
    
    Returns:
    poly_regr: modelo de transformação polinomial a ser aplicado posteriormente, permitindo ao modelo fazer previsões;
    model: modelo de regressão linear para fazer as previsões.
    """
    # Separando as variáveis:
    X = city_data[['day_count']]
    Y = city_data[['deaths']]
    
    # Aplicando a transformação polinomial aos dados:
    poly_regr = PolynomialFeatures(degree = n_degree)
    X_poly = poly_regr.fit_transform(X)
    
    # Criando dados de treinos e de teste:
    X_poly_train, X_poly_test, Y_train, Y_test = train_test_split(X_poly, Y,
                                                                 test_size = 0.2,
                                                                 random_state = seed)
    
    # Criando o modelo:
    model = LinearRegression()
    
    # Treinando o modelo:
    model.fit(X_poly_train, Y_train)
    
    # Fazendo as previsões para os dados de teste:
    Y_pred = model.predict(X_poly_test)
    
    # Avaliando o modelo pela métrica MAE (mean absolut error):
    mae = round(mean_absolute_error(Y_test, Y_pred))
    
    sns.scatterplot(x = 'day_count', y = 'deaths', data = city_data).set(
        xlabel = 'Contagem dos dias a partir de ' + min_date,
        ylabel = 'Número de mortos',
        title = 'Evolução Mortes x Regressão Polinomial de Grau ' + str(n_degree))
    plt.plot(X, model.predict(X_poly), color = 'r')
    plt.show()
    
    print('O erro médio absoluto da previsão do número de mortos para a regressão polinomial de grau ' +
          str(n_degree) + ' é de: ' + str(mae) + '.')
 
    return poly_regr, model

<p>
    Abaixo, vamos criar um modelo de regressão para a cidade de <b>São Paulo</b>. O algoritmo utilizado será o <b>LinearRegression</b> juntamente com o algoritmo <b>PolynomialFeatures</b>, ambos do pacote Scikit-Learn - como pode ser observado na função criada anteriormente.
</p>
<p>
    Esta mesma função será aplicada para criar os modelos de previsão para as demais capitais de interesse.
</p>

<h3 id = modelosp>São Paulo - SP</h3>

In [ ]:
# Filtrando os dados para a cidade de São Paulo:
covid_sp = city_filter('São Paulo - SP')

# Avaliando o crescimento no número de mortes na capital paulista:
deaths_evolution(covid_sp, 'São Paulo - SP')

In [ ]:
# Criação e avaliação do modelo de regressão para a cidade de São Paulo:
sp_poly_regr_5, sp_model_5 = covid_poly_model(covid_sp, n_degree = 5, seed = 200)

sp_poly_regr_6, sp_model_6 = covid_poly_model(covid_sp, n_degree = 6, seed = 200)

sp_poly_regr_7, sp_model_7 = covid_poly_model(covid_sp, n_degree = 7, seed = 200)

<p>
    O crescimento do número de mortes na cidade de <b>São Paulo</b> não segue uma taxa linear. Este mesmo perfil é esperado nas demais cidades e, por isso, o modelo de regressão linear foi construído com uma transformação polinomial da variável preditora. </p>
<p>
    O <i>erro absoluto médio</i> não foi muito discrepante nos três modelos testados. No entanto, observando a curva vermelha (que representa o modelo de regressão polinomial criado), aquela que melhor parece acompanhar a tendência de crescimento dos últimos dias e também apresentou o menor erro médio nas previsões é a curva relativa ao modelo de grau 5, sendo este assumido, portanto, como o grau que melhor respresenta o perfil da curva de crescimento estudada:
</p>

In [ ]:
sp_poly_regr = sp_poly_regr_5
sp_model = sp_model_5

<p>
    Na seqüência, vamos repetir o procedimento para criação de um modelo preditivo para as cidades do <b>Rio de Janeiro</b>, <b>Belém</b> e <b>Porto Alegre</b>:
</p>

<h3 id = modelorj>Rio de Janeiro - RJ</h3>

In [ ]:
# Filtrando os dados para a cidade do Rio de Janeiro:
covid_rj = city_filter('Rio de Janeiro - RJ')

# Avaliando o crescimento no número de mortes na capital carioca:
deaths_evolution(covid_rj, 'Rio de Janeiro - RJ')

In [ ]:
# Criação e avaliação do modelo de regressão para a cidade do Rio de Janeiro:
rj_poly_regr_4, rj_model_4 = covid_poly_model(covid_rj, n_degree = 4, seed = 201)

rj_poly_regr_5, rj_model_5 = covid_poly_model(covid_rj, n_degree = 5, seed = 201)

rj_poly_regr_6, rj_model_6 = covid_poly_model(covid_rj, n_degree = 6, seed = 201)

<p>
    Os três modelos de graus 5 e 6 apresentam <i>mean absolut error</i> iguais. No entanto, a regressão polinomial de grau 5 parece representar melhor o crescimento do número de mortos nos últimos dias e será, portanto, o modelo adotado para fazermos as previsões.
</p>

In [ ]:
rj_poly_regr = rj_poly_regr_5
rj_model = rj_model_5

<h3 id = modelobl>Belém - PA</h3>

In [ ]:
# Filtrando os dados para a cidade de Belém:
covid_bl = city_filter('Belém - PA')

# Avaliando o crescimento no número de mortes na capital paraense:
deaths_evolution(covid_bl, 'Belém - PA')

In [ ]:
# Criação e avaliação do modelo de regressão para a cidade de Belém:
bl_poly_regr_4, bl_model_4 = covid_poly_model(covid_bl, n_degree = 4, seed = 202)

bl_poly_regr_5, bl_model_5 = covid_poly_model(covid_bl, n_degree = 5, seed = 202)

bl_poly_regr_6, bl_model_6 = covid_poly_model(covid_bl, n_degree = 6, seed = 202)

<p>
    Novamente, pelo perfil das curvas apresentadas e pelo avaliação de erros do modelo, o modelo de regressão de grau 5 será o adotado para fazermos as previsões para a cidade de Belém:
</p>

In [ ]:
bl_poly_regr = bl_poly_regr_4
bl_model = bl_model_4

<h3 id = modelopa>Porto Alegre - RS</h3>

In [ ]:
# Filtrando os dados para a cidade de Porto Alegre:
covid_pa = city_filter('Porto Alegre - RS')

# Avaliando o crescimento no número de mortes na capital gaúcha:
deaths_evolution(covid_pa, 'Porto Alegre - RS')

<p>
    Entre as capitais estudadas, <b>Porto Alegre</b> parece ter um crescimento de óbitos mais próximo a um perfil linear, ainda assim, usaremos uma aproximação polinomial para melhor representar a evolução dos casos de morte pela Covid-19 na cidade.
</p>

In [ ]:
# Criação e avaliação do modelo de regressão para a cidade de Porto Alegre:
pa_poly_regr_3, pa_model_3 = covid_poly_model(covid_pa, n_degree = 3, seed = 203)

pa_poly_regr_4, pa_model_4 = covid_poly_model(covid_pa, n_degree = 4, seed = 203)

pa_poly_regr_5, pa_model_5 = covid_poly_model(covid_pa, n_degree = 5, seed = 203)

<p>
    Novamente, os três modelos testados apresentam desempenho muito similar. No entanto, o de grau 4 parece acompanhar melhor a tendência de crescimento no número de mortos nos últimos dias:
</p>

In [ ]:
pa_poly_regr = pa_poly_regr_4
pa_model = pa_model_4

<p>
    Na seqüência, utilizaremos os modelos criados para prever o número total de mortes contabilizado em dias futuros. 
</p>
<p>
    <b>Importante</b>: cabe ressaltar mais uma vez que a única variável preditora deste estudo é a data de referência, sendo que outros fatores que interferem diretamente no número de mortes, tais como número de leitos disponíveis, perfil social da população, taxa de adesão ao distanciamento social, medidas tomadas pelo governo local, entre outros, não estão sendo diretamente considerados. Sendo assim, esta projeção de mortes não é sensível a possíveis alterações nestes fatores que definem a evolução da doença em cada uma das cidades que são objeto deste estudo.
</p>

<h2 id = prev>5- Previsão de mortes por Covid-19 nas cidades estudadas</h2>

In [ ]:
# Carregando o arquivo com as datas futuras de referência e aplicando as transformações necessárias:
pred_df = pd.read_csv('../input/covid19-br/covid_predict.csv', parse_dates = ['date'])
pred_df['day_count'] = (pred_df['date'] - min(covid_cap_br['date']))
pred_df['day_count']

In [ ]:
# Transformando a contagem de dias em números inteiros:
pred_df['day_count'] = pd.to_numeric(pred_df['day_count'].dt.days, downcast='integer')

In [ ]:
# Criando uma função para facilitar a aplicação dos modelos criados anteriormente:
def make_prediction(model, poly_regr, cidade):
    """Utiliza os modelos de transformação polinomial e regressão linear criados anteriormente para fazer previsões.
    
    Args:
    model: modelo de regressão linear criado pela função covid_poly_model();
    poly_regr: modelo de transformação polinomial criado pela função covid_poly_model();
    cidade: string da cidade que é objeto da previsão.
    
    Returns:
    retorna duas strings com a previsão do número de mortos total até as duas datas contidas no objeto 'pred_df'.
    """
    Y_predicted = model.predict(poly_regr.fit_transform([[pred_df['day_count'][0]], [pred_df['day_count'][1]]]))
    
    print('Até ' + pred_df['date'][0].strftime('%d/%m/%Y') + ', na cidade de ' + cidade + ', a previsão é de ' +
      str(round(int(Y_predicted[0]))) + ' mortos por Covid-19.')
    
    print('Até ' + pred_df['date'][1].strftime('%d/%m/%Y') + ', na cidade de ' + cidade + ', a previsão é de ' +
      str(round(int(Y_predicted[1]))) + ' mortos por Covid-19.')

<h3 id = prevsp>Previsões para São Paulo - SP</h3>

In [ ]:
# Fazendo as previsões para a cidade de São Paulo:
make_prediction(sp_model, sp_poly_regr, 'São Paulo - SP')

<h3 id = prevrj>Previsões para Rio de Janeiro - RJ</h3>

In [ ]:
# Fazendo as previsões para a cidade do Rio de Janeiro:
make_prediction(rj_model, rj_poly_regr, 'Rio de Janeiro - RJ')

<h3 id = prevbl>Previsões para Belém - PA</h3>

In [ ]:
# Fazendo as previsões para a cidade de Belém:
make_prediction(bl_model, bl_poly_regr, 'Belém - PA')

<h3 id = prevpa>Previsões para Porto Alegre - RS</h3>

In [ ]:
# Fazendo as previsões para a cidade de Porto Alegre:
make_prediction(pa_model, pa_poly_regr, 'Porto Alegre - RS')

<h2 id = conclusao>6- Conclusão</h2>

<p>
    As cidades estudadas apresentam perfis diferentes de evolução do número de mortes, sendo que a grande variação diária de notificações não permite prever com maior precisão a tendência dos próximos dias. No entanto, como panorama geral, é possível observar que existe uma tendência crescente de mortes, mesmo em cidades com situação mais confortável. Não é possível afirmar se o país de um modo geral já chegou ao pico da pandemia, no entanto, pode-se afirmar que o país não entrou na fase de redução de casos confirmados ou de mortes.
</p>
<p>
    Desta forma, nos resta apenas continuar seguindo as recomendações estabelecidas pelos órgãos técnicos responsáveis que, em sua grande maioria, recomendam o distanciamento social, a higienização constante das mãos, uso de máscaras de proteção, entre outros.
</p>

### Fim.